# DICOM to OMOP: upload the custom vocabularies

This notebook demonstrates how you can upload the DICOM custom vocabularies to SQL database. We are using pyodbc, but you can run the SQL section of your code directly your SQL interface. The files can be found under "files" folder, and if you want to learn more about how we identified DICOM terminologies for integration, please refer to the "transform_dicom_to_omop" notebook.

* DDL & DML Scripts using Python & SQL
   1. Update `VOCABULARY` table
   2. Update `CONCEPT_CLASS` table
   3. Update `CONCEPT` table
   4. Update `CONCEPT_RELATIONSHIP` table

In [ ]:
# Connect to you database
import psycopg2

conn = psycopg2.connect(
    database="",
    user="",
    password="",
    host="",
    port="",
    connect_timeout = 6000
)

cursor = conn.cursor()

In [ ]:
# Update VOCABULARY
sql = '''
    INSERT INTO dbo.VOCABULARY (vocabulary_id, vocabulary_name, vocabulary_reference, vocabulary_version, vocabulary_concept_id)
    VALUES ('DICOM', 'Digital Imaging and Communications in Medicine (National Electrical Manufacturers Association)',  'https://www.dicomstandard.org/current', 'NEMA Standard PS3', 2128000000)
    '''
cursor.execute(sql)
conn.commit()

In [ ]:
# Update CONCEPT_CLASS
sql = '''
    INSERT INTO CONCEPT_CLASS (concept_class_id, concept_class_name, concept_class_concept_id)
    VALUES ('DICOM Attributes', 'DICOM Attributes', 2128000001),
           ('DICOM Value Sets', 'DICOM Value Sets', 2128000002)
    '''
cursor.execute(sql)
conn.commit()

In [ ]:
# Import the file that contain DICOM custom concepts
import pandas as pd

omop_table_staging = pd.read_csv('./files/OMOP CDM Staging/omop_table_staging_v2.csv')

In [ ]:
# Update CONCEPT
# Load the file for DICOM attributes and value sets
from datetime import datetime

sql = '''
    INSERT INTO dbo.concept (concept_id, concept_name, domain_id, vocabulary_id, concept_class_id, concept_code, valid_start_date, valid_end_date) 
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
    '''
for index, row in omop_table_staging.iterrows():
    # Convert the valid_start_date and valid_end_date from integer (YYYYMMDD) to date
    valid_start_date = datetime.strptime(str(row['valid_start_date']), '%Y%m%d').date() if pd.notnull(row['valid_start_date']) else None
    valid_end_date = datetime.strptime(str(row['valid_end_date']), '%Y%m%d').date() if pd.notnull(row['valid_end_date']) else None

    cursor.execute(sql, (
        row['concept_id'], 
        row['concept_name'], 
        row['domain_id'], 
        row['vocabulary_id'], 
        row['concept_class_id'], 
        row['concept_code'], 
        valid_start_date, 
        valid_end_date
    ))

conn.commit()

In [ ]:
# check the upload
sql = "select * from dbo.concept where vocabulary_id = 'DICOM'"
omop_dicom_test = pd.read_sql_query(sql, conn)
omop_dicom_test.head()

In [ ]:
# import DICOM concept_relationship source file 
import pandas as pd

# 1. Attributes-CID-ValueSets
concept_relationship_staging = pd.read_pickle('./files/OMOP CDM Staging/part3_to_part16_relationship_via_CID.pkl')
# 2. Attributes-Code String values
cs_values_maps_to_value = pd.read_csv('./files/OMOP CDM Staging/cs_values_maps_to_value.csv')
# 3. DICOM Code String to OMOP Standard coding systems
cs_values_maps_to = pd.read_csv('./files/OMOP CDM Staging/cs_values_maps_to.csv')

In [ ]:
# Update CONCEPT_RELATIONSHIP
from datetime import datetime

sql = '''
    INSERT INTO dbo.concept_relationship (concept_id_1, concept_id_2, relationship_id, valid_start_date, valid_end_date) 
    VALUES (%s,%s,%s,%s,%s)
    '''
for index, row in concept_relationship_staging.iterrows():
    # Convert the valid_start_date and valid_end_date from integer (YYYYMMDD) to date
    valid_start_date = datetime.strptime(str(row['valid_start_date']), '%Y%m%d').date() if pd.notnull(row['valid_start_date']) else None
    valid_end_date = datetime.strptime(str(row['valid_end_date']), '%Y%m%d').date() if pd.notnull(row['valid_end_date']) else None

    cursor.execute(sql, (
        row['concept_id_1'], 
        row['concept_id_2'], 
        row['relationship_id'], 
        valid_start_date, 
        valid_end_date
    ))

conn.commit() 

In [ ]:
# Update CONCEPT_RELATIONSHIP

sql = '''
    INSERT INTO dbo.concept_relationship (concept_id_1, concept_id_2, relationship_id, valid_start_date, valid_end_date) 
    VALUES (%s,%s,%s,%s,%s)
    '''
for index, row in cs_values_maps_to_value.iterrows():
    valid_start_date = datetime.strptime(str(19930101), '%Y%m%d').date()
    valid_end_date = datetime.strptime(str(20991231), '%Y%m%d').date()

    cursor.execute(sql, (
        row['concept_id_1'], 
        row['concept_id_2'], 
        row['relationship_id'], 
        valid_start_date, 
        valid_end_date
    ))

conn.commit()

In [ ]:
# Update CONCEPT_RELATIONSHIP

sql = '''
    INSERT INTO dbo.concept_relationship (concept_id_1, concept_id_2, relationship_id, valid_start_date, valid_end_date) 
    VALUES (%s,%s,%s,%s,%s)
    '''
for index, row in cs_values_maps_to.iterrows():
    valid_start_date = datetime.strptime(str(19930101), '%Y%m%d').date()
    valid_end_date = datetime.strptime(str(20991231), '%Y%m%d').date()

    cursor.execute(sql, (
        row['concept_id_1'], 
        row['concept_id_2'], 
        row['relationship_id'], 
        valid_start_date, 
        valid_end_date
    ))

conn.commit()

In [ ]:
# close the cursor and connection
cursor.close()
conn.close()